## RandSubWare

> By deut-erium
>
> \[Insert gpt-generated sampletext description here\]
>
> `nc chals.sekai.team 3037`
>
> Attachment: chall.py

We get an encryption oracle of a toy substitution-permutation network cipher with a randomly generated S-box and need to obtain the first round key.

This 5 round SPN cipher has poor diffusion and is vulnerable to differential analysis. Obtain a corpus by sending pairs of plaintext which differ by one block at a time. Guess the last round key 6-bits at a time, then undo the last key XOR and substitution step prior to that. Correctly guessing the round key will mean the two ciphertexts will have low hamming distance from each other. Once all of the last round key has been obtained, the key expansion can be reversed to obtain the first round key.

In [1]:
from pwn import *
from randsubware import gen_pbox, SPN
import random
from Crypto.Util.number import bytes_to_long

r = remote("chals.sekai.team", 3037)
r.recvuntil(b"sbox: ")
sbox = list(bytes.fromhex(r.recvline().strip().decode()))

box_size = 6
num_box = 16
pbox = gen_pbox(box_size, num_box)

spnkey = 0 # dummy key
# local instance of SPN to help reversing steps
spn = SPN(sbox, pbox, spnkey, 5)

# Generate plaintexts with one block differenece
differential_inputs = []
for _ in range(25_000):
    head = random.randrange(0, 2**90) * 64
    da, db = random.sample(range(64), k=2)
    differential_inputs.append((head + da, head + db))
payload = []
r.sendline(b"1")
for a, b in differential_inputs:
    payload.append(f"{a:012x}")
    payload.append(f"{b:012x}")
r.sendline("".join(payload).encode())

# Get ciphertextx
r.recvuntil(b"Quota remaining:")
r.recvline()
bulk_output = r.recvline().strip().decode()
differential_outputs = []
for i in range(0, len(bulk_output), 48):
    differential_outputs.append((
        bytes_to_long(bytes.fromhex(bulk_output[i:i+24])),
        bytes_to_long(bytes.fromhex(bulk_output[i+24:i+48]))
    ))

[x] Opening connection to chals.sekai.team on port 3037
[x] Opening connection to chals.sekai.team on port 3037: Trying 34.148.151.228
[+] Opening connection to chals.sekai.team on port 3037: Done


In [2]:
from tqdm.auto import tqdm
from randsubware import rotate_left

def solve_last_key(block_i):
    key_scores = []
    # For each possible key
    for guess_key in range(64):
        # Total hamming differences across all pairs of differential ciphertexts
        total_differential = 0
        for a, b in differential_outputs:
            a_block = (a >> (block_i * 6)) & 63
            b_block = (b >> (block_i * 6)) & 63

            # Undo xossub steps
            a_unxorsub = spn.SINV[a_block ^ guess_key]
            b_unxorsub = spn.SINV[b_block ^ guess_key]

            # Calculate hamming difference
            total_differential += (a_unxorsub ^ b_unxorsub).bit_count()
        key_scores.append((total_differential, guess_key))

    # Find key with best fit
    key_scores.sort()
    return key_scores[0][1]

# Get the last key
print()
last_key = []
for block_i in tqdm(range(16)):
    last_key.append(solve_last_key(block_i))
print("last key blocks", last_key)
last_key = int("".join(f"{i:06b}" for i in last_key[::-1]), 2)
print("last key", last_key)

first_key = last_key
for _ in range(5):
    first_key = rotate_left(
        spn.inv_sbox(first_key), 96 - box_size - 1, 96
    )
print("first key", first_key)

r.sendline(f"2\n{first_key}".encode())
print(r.recvuntil(b"}"))

  0%|          | 0/16 [00:00<?, ?it/s]

last key blocks [59, 10, 27, 18, 11, 61, 54, 2, 43, 62, 52, 62, 33, 59, 25, 13]
last key 16594782968362502945933734587
first key 51786547602721728170223252735
b'Choose API option\n1. Test encryption\n2. Get Flag\n(int) key: SEKAI{d04e4ba19456a8a42584d5913275fe68c30893b74eb1df136051bbd2cd086dd0}'
